In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [73]:
h100_subcharts = pd.read_csv('hot100_radio_streaming_sales_Kpop.csv')
wdss = pd.read_csv('wdss.csv')

## create standardized search_title column for wdss to match other set
wdss['search_title'] = wdss['song_title'].str.replace(r"(\(.*\))", "")
wdss['search_title'] = wdss['search_title'].str.strip()

##match wdss processing to h100/subcharts processing
#clean spacing: add space before linebreaks (so theyre counted separately)
wdss["translated_lyrics"] = wdss["translated_lyrics"].str.replace(r"(\n+)", " \n")
wdss["translated_lyrics"] = wdss["translated_lyrics"].str.replace(r"(\s+)", " ") #restandardize whitespace
wdss["translated_lyrics"] = wdss["translated_lyrics"].str.replace(r"(\s[, | \' | \? | \! | \. | \-]+)", " ")  #delete hanging punctuation

wdss["original_lyrics"] = wdss["original_lyrics"].str.replace(r"(\n+)", " \n")
wdss["original_lyrics"] = wdss["original_lyrics"].str.replace(r"(\s+)", " ") #restandardize whitespace
wdss["original_lyrics"] = wdss["original_lyrics"].str.replace(r"(\s[, | \' | \? | \! | \. | \-]+)", " ")  #delete hanging punctuation

In [74]:
## redo word classifying and sorting to focus on tokenized words instead of the whole string

# define function to classify words as roman/presumably english if it contains a roman letter
def is_roman(word):
    if re.search("[A-Z | a-z ]", word):
        return True
    else:
        return False

#test to make sure it classifies words as desired
#we will count numeric characters as part of "original" because often they are pronounced in korean
# ie, '2!3!' is sung as 'dul! set!'
print(is_roman('h1'))
print(is_roman('h'))
print(is_roman('1'))
print(is_roman('거짓이'))
print(is_roman('h거짓이'))

#there are a few cases where roman words and non-roman words are smushed together, but they are rare and we'll still count them as roman/english

True
True
False
False
True


In [75]:
#redo counting english words - wdss
for i, r in wdss.iterrows():

    #tokenize words
    this_original = r['original_lyrics'].split(' ')
    this_english = [] #list to store words classified as roman/english (in some cases though these could be spanish or french, so)
    #determine if word contains roman characters or not
    for word in this_original:
        if is_roman(word):
            this_english.append(word)

    #combine words into 1 string
    this_english = ' '.join(this_english)
    wdss.at[i, 'english_lyrics'] = this_english

#redo counting english words - h100_subcharts
for i, r in h100_subcharts.iterrows():

    #tokenize words
    this_original = r['original_lyrics'].split(' ')
    this_english = [] #list to store words classified as roman/english (in some cases though these could be spanish or french, so)
    #determine if word contains roman characters or not
    for word in this_original:
        if is_roman(word):
            this_english.append(word)

    #combine words into 1 string
    this_english = ' '.join(this_english)
    h100_subcharts.at[i, 'english_lyrics'] = this_english

In [76]:
##count number of english lyrics and total lyrics
wdss['english_count'] = [len(str(lyr).split(" ")) for lyr in wdss['english_lyrics']]
wdss['total_count'] = [len(str(lyr).split(" ")) for lyr in wdss['original_lyrics']]

### create columns for english percentage
h100_subcharts['english_percentage'] = h100_subcharts['english_count'] / h100_subcharts['total_count']
wdss['english_percentage'] = wdss['english_count'] / wdss['total_count']

In [77]:
##test on one song to see if they match
test_title = 'Life Goes On'
h100_value = h100_subcharts.loc[h100_subcharts['search_title'] == test_title]['english_percentage']
wdss_value = wdss.loc[wdss['search_title'] == test_title]['english_percentage']
print('Song ', test_title)
print(h100_value, 'perc. english in h100/subcharts')
print(wdss_value, 'perc. english in wdss')
print('ENGLISH COUNTS:')
print(h100_subcharts.loc[h100_subcharts['search_title'] == test_title]['english_count'], 'h100/subcharts')
print(wdss.loc[wdss['search_title'] == test_title]['english_count'], 'wdss')
print('TOTAL COUNTS:')
print(h100_subcharts.loc[h100_subcharts['search_title'] == test_title]['total_count'], 'h100/subcharts')
print(wdss.loc[wdss['search_title'] == test_title]['total_count'], 'wdss')

Song  Life Goes On
29    0.34981
Name: english_percentage, dtype: float64 perc. english in h100/subcharts
1230    0.346008
Name: english_percentage, dtype: float64 perc. english in wdss
ENGLISH COUNTS:
29    92
Name: english_count, dtype: int64 h100/subcharts
1230    91
Name: english_count, dtype: int64 wdss
TOTAL COUNTS:
29    263
Name: total_count, dtype: int64 h100/subcharts
1230    263
Name: total_count, dtype: int64 wdss


In [78]:
### verify lyrics for both datsets look correct for test song
for i, r in h100_subcharts.iterrows():
    if r['search_title'] == test_title:
        print(r['english_lyrics'])

yeah oh Like an echo in the forest Yeah, life goes on Like an arrow in the blue sky On my pillow, on my table Yeah, life goes on Like this again (Ayy) (No) Mm-mm-mm-mm (Ooh, ooh, ooh, oh-woah) oh (Oh, oh, oh) Like an echo in the forest Yeah, life goes on (Oh, woah) Like an arrow in the blue sky On my pillow, on my table Yeah, life goes on Like this again I remember I, yeah-yeah-yeah-yeah I remember Ah-ah, ah-ah I remember I, yeah-yeah-yeah-yeah I remember Ah-ah, ah-ah


In [79]:
for i, r in wdss.iterrows():
    if r['search_title'] == test_title:
        print(r['english_lyrics'])

yeah oh Like an echo in the forest Yeah, life goes on Like an arrow in the blue sky On my pillow, on my table Yeah, life goes on Like this again (Ayy) (No) Mm-mm-mm-mm (Ooh, ooh, ooh, oh-woah) oh (Oh, oh, oh) Like an echo in the forest Yeah, life goes on (Oh, woah) Like an arrow in the blue sky On my pillow, on my table Yeah, life goes on Like this again I remember I, yeah-yeah-yeah-yeah I remember Ah-ah, ah-ah I remember I, yeah-yeah-yeah-yeah I remember Ah-ah, ah-ah


In [80]:
#### identify all songs in both datasets and make sure they have matching lyrics info
common_songs = h100_subcharts.merge(wdss, how='inner', on=['search_title'])

In [81]:
target_col = 'total_count'
common_songs.loc[common_songs[target_col+'_y'] != common_songs[target_col+'_x']][['search_title', 'artist_x', 'artist_y', target_col+'_y', target_col+'_x']]

,search_title,artist_x,artist_y,total_count_y,total_count_x
6,Fake Love,BTS,BTS,409,408
9,Idol,BTS feat. Nicki Minaj,BTS,356,430
14,Black Swan,BTS,BTS,379,378
16,On,BTS,BTS,410,409
21,Dynamite,BTS,VIXX,286,469
27,Stay,BTS,Taeyeon,105,234
28,Stay,BTS,BTS,235,234
37,Home,BTS,Seventeen,272,412
38,Home,BTS,BTS,413,412
40,Dionysus,BTS,BTS,514,513


In [82]:
### verify lyrics for both datsets look correct for test song
for i, r in h100_subcharts.iterrows():
    if r['search_title'] == 'Ugh!':
        print(r['original_lyrics'])

Let's go, yeah, yeah, yeah 타닥 또 타오르는 저 불씨 기름에 닿기 전에 먼저 집어삼키네 필시 휩쓸려가겠지 예 예 음 오늘의 선수 입장하시네 건수를, yeah 물기 시작하면 둥둥둥 동네북이 돼 둥둥둥 툭툭 건드네 괜시리 툭툭, yeah 반응이 없음 걍 담궈버리지 푹푹, yeah 진실도 거짓이 돼 거짓도 진실이 돼 이곳에선 모두가 도덕적 사고와 판단이 완벽한 사람이 돼 웃기시네 분노? 물론 필요하지 타오를 땐 이유가 있어 어쩌면 우리의 역사지 그게 세상을 바꾸기도 하지 But 이건 분노 아닌 분뇨 뭐가 분노인지 you know? 분노인 척하며 죽여 진짜 분노 질려버린 수도 없이 많은 people 넌 나만 죽이는 게 아니야 (아니야) 똥 밟는 게 익숙해 우리야 (우리야) 무감각해진 저 사람들 봐 (봐) 분뇨, 무관심 너넨 팀이야, yeah (19115977) 나는 악의에 가득 찬 분노에 분노해 나는 악의에 가득 찬 분노에 분노해 나는 욱해 욱해 나는 욱해 욱해 나는 꺼져야만 했던 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 (Yeah, yeah) 그래 욱 욱 욱해라 욱 재가 될 때까지 그래 욱해라 욱 그래 욱 욱 욱해라 욱 부러질 때까지 그래 욱해라 욱 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 이 세상 분노가 지배함 분노가 없음 다 못 사나 봐 분노하고 또 분노하고 분노하고 그리 미쳐가고 욱 욱 욱 욱 분노하는 이유도 다 수만 가지 선의와 악의도 다 매한가지 분노할 수 있다만 남의 삶에 피해가 있는 건 I don’t like 그건 stop, ayy 누구의 행동에 누구는 아파해 누구의 언행에 누구는 암담해 누구의 찰나에 누구 순간이 돼 누구의 분노에 누구 목숨이 돼 썩을 퉤 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 아 대체 욕 좀 먹는 게 왜 잘 벌잖아 또 징징대 왜 그 정돈 감수해야지 에헴 에헴 에헴 에헴 에헴

In [83]:
### verify lyrics for both datsets look correct for test song
for i, r in wdss.iterrows():
    if r['search_title'] == 'Ugh!':
        print(r['original_lyrics'])

Let's go, yeah, yeah, yeah 타닥 또 타오르는 저 불씨 기름에 닿기 전에 먼저 집어삼키네 필시 휩쓸려가겠지 예 예 음 오늘의 선수 입장하시네 건수를, yeah 물기 시작하면 둥둥둥 동네북이 돼 둥둥둥 툭툭 건드네 괜시리 툭툭, yeah 반응이 없음 걍 담궈버리지 푹푹, yeah 진실도 거짓이 돼 거짓도 진실이 돼 이곳에선 모두가 도덕적 사고와 판단이 완벽한 사람이 돼 웃기시네 분노? 물론 필요하지 타오를 땐 이유가 있어 어쩌면 우리의 역사지 그게 세상을 바꾸기도 하지 But 이건 분노 아닌 분뇨 뭐가 분노인지 you know? 분노인 척하며 죽여 진짜 분노 질려버린 수도 없이 많은 people 넌 나만 죽이는 게 아니야 (아니야) 똥 밟는 게 익숙해 우리야 (우리야) 무감각해진 저 사람들 봐 (봐) 분뇨, 무관심 너넨 팀이야, yeah (19115977) 나는 악의에 가득 찬 분노에 분노해 나는 악의에 가득 찬 분노에 분노해 나는 욱해 욱해 나는 욱해 욱해 나는 꺼져야만 했던 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 (Yeah, yeah) 그래 욱 욱 욱해라 욱 재가 될 때까지 그래 욱해라 욱 그래 욱 욱 욱해라 욱 부러질 때까지 그래 욱해라 욱 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 이 세상 분노가 지배함 분노가 없음 다 못 사나 봐 분노하고 또 분노하고 분노하고 그리 미쳐가고 욱 욱 욱 욱 분노하는 이유도 다 수만 가지 선의와 악의도 다 매한가지 분노할 수 있다만 남의 삶에 피해가 있는 건 I don’t like 그건 stop, ayy 누구의 행동에 누구는 아파해 누구의 언행에 누구는 암담해 누구의 찰나에 누구 순간이 돼 누구의 분노에 누구 목숨이 돼 썩을 퉤 나는 욱해 욱해 나는 욱해 욱해 나는 악의에 가득 찬 분노에 분노해 나는 꺼져야만 했던 그 분노에 분노해 아 대체 욕 좀 먹는 게 왜 잘 벌잖아 또 징징대 왜 그 정돈 감수해야지 에헴 에헴 에헴 에헴 에헴

In [84]:
# wrong lyrics were identified for gangnam style in wdss....
# h100 is idoll ft. nicki minaj, wdss is idol original version, but they should both be nicki minaj version
# h100 has correct all night lyrics, wdss incorrectly matched it to glory by rm ':|
# ugh! is onlly off by a few words for each dataset so I will let it go

##so basically h100 had correct versions, will overwrite wdss to that

# would be good to do random checks for other wdss songs if time since mismatched songs obviously happened
# (redo it now that ive figured out more accurate way of using genius search??)

In [85]:
## overwrite wdss mistakes with h100 info
to_correct = list(common_songs.loc[common_songs[target_col+'_y'] != common_songs[target_col+'_x']]['search_title'])

for song in to_correct:
    print(song)
    #locate song in each dataset
    h100_index = h100_subcharts.loc[h100_subcharts['search_title'] == song].index[0]
    wdss_index = wdss.loc[wdss['search_title'] == song].index[0]

    ##overwrite wdss info with h100 info
    wdss.at[wdss_index, 'original_lyrics'] = h100_subcharts.at[h100_index, 'original_lyrics']
    wdss.at[wdss_index, 'english_lyrics'] = h100_subcharts.at[h100_index, 'english_lyrics']
    wdss.at[wdss_index, 'translated_lyrics'] = h100_subcharts.at[h100_index, 'translated_lyrics']
    wdss.at[wdss_index, 'english_count'] = h100_subcharts.at[h100_index, 'english_count']
    wdss.at[wdss_index, 'total_count'] = h100_subcharts.at[h100_index, 'total_count']
    wdss.at[wdss_index, 'english_percentage'] = h100_subcharts.at[h100_index, 'english_percentage']

Fake Love
Idol
Black Swan
On
Dynamite
Stay
Stay
Home
Home
Dionysus
All Night
All Night
Heartbeat


In [94]:
### correct wikipedia mistake, add gangnam style to radio chart

gangnam_i = h100_subcharts.loc[h100_subcharts['search_title'] == 'Gangnam Style'].index[0]
h100_subcharts.at[gangnam_i, 'peak_position_radio'] = 12
h100_subcharts.at[gangnam_i, 'chart_date_radio'] = '2012-10-26'
h100_subcharts.loc[h100_subcharts['search_title'] == 'Gangnam Style'][['peak_position_radio', 'chart_date_radio']]

,peak_position_radio,chart_date_radio
2,12.0,2012-10-26


In [70]:
## overwrite each csv with the updated datasets
h100_subcharts.to_csv('hot100_radio_streaming_sales_Kpop.csv')
wdss.to_csv('wdss.csv')